# Modelling the Data

## Trading Strategy Using CSV Data
# 
This notebook demonstrates how to load historical stock data from a CSV file, create lagged return features, train a linear regression model, evaluate performance, and simulate a simple trading strategy.

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit, train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.stattools import adfuller
import traceback
from tabulate import tabulate

## Load Data

In [31]:
def load_data(csv_file):
    """Loads data from CSV file with error handling"""
    try:
        print(f"Loading data from {csv_file}...")
        data = pd.read_csv(csv_file, parse_dates=['Date'], index_col='Date')
        data.sort_index(inplace=True)
        print("Data loaded successfully")
        print(f"Data shape: {data.shape}")
        print(f"Data range: {data.index.min()} to {data.index.max()}")
        return data
    except Exception as e:
        print(f"Error loading data: {e}")
        return None

# Feature Engineering

In [32]:
def engineer_features(df, diff_order=1, lookback_periods=30):
    """Engineers financial features with technical indicators and lagged features"""
    try:
        if df is None:
            raise ValueError("Input dataframe is None")
        
        print("\n=== Initial Data ===")
        print(f"Shape: {df.shape}")
        print(f"NaN Counts:\n{df.isna().sum()}")

        # Validate input dataframe
        required_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
        missing = [col for col in required_columns if col not in df.columns]
        if missing:
            raise ValueError(f"Missing required columns: {missing}")

        # Convert Volume to numeric
        df['Volume'] = pd.to_numeric(df['Volume'], errors='coerce').fillna(0)

        # Basic price transformations
        df['Close_diff'] = df['Close'].diff(diff_order).ffill()
        print("\n=== After Close_diff ===")
        print(f"NaN Counts:\n{df.isna().sum()}")

        df['Return'] = df['Close'].pct_change().fillna(0)
        print("\n=== After Return ===")
        print(f"NaN Counts:\n{df.isna().sum()}")

        df['Range'] = (df['High'] - df['Low']) / df['Close'].shift(1).bfill()
        print("\n=== After Range ===")
        print(f"NaN Counts:\n{df.isna().sum()}")

        # Moving Averages
        for window in [5, 10, 20, 50, 200]:
            df[f'SMA_{window}'] = df['Close'].rolling(window, min_periods=1).mean()
            df[f'EMA_{window}'] = df['Close'].ewm(span=window, adjust=False).mean()
        print("\n=== After Moving Averages ===")
        print(f"NaN Counts:\n{df.isna().sum()}")

        # Bollinger Bands
        df['BB_MA20'] = df['Close'].rolling(20, min_periods=1).mean()
        df['BB_Upper'] = df['BB_MA20'] + 2 * df['Close'].rolling(20).std()
        df['BB_Lower'] = df['BB_MA20'] - 2 * df['Close'].rolling(20).std()
        df['BB_Width'] = (df['BB_Upper'] - df['BB_Lower']) / df['BB_MA20']
        print("\n=== After Bollinger Bands ===")
        print(f"NaN Counts:\n{df.isna().sum()}")

        # RSI (14-day)
        delta = df['Close'].diff()
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        
        avg_gain = gain.rolling(14, min_periods=1).mean()
        avg_loss = loss.rolling(14, min_periods=1).mean()
        
        rs = avg_gain / (avg_loss + 1e-10)  # Prevent division by zero
        df['RSI_14'] = 100 - (100 / (1 + rs))
        print("\n=== After RSI ===")
        print(f"NaN Counts:\n{df.isna().sum()}")

        # MACD
        ema12 = df['Close'].ewm(span=12, adjust=False).mean()
        ema26 = df['Close'].ewm(span=26, adjust=False).mean()
        df['MACD'] = ema12 - ema26
        df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
        df['MACD_Hist'] = df['MACD'] - df['MACD_Signal']
        print("\n=== After MACD ===")
        print(f"NaN Counts:\n{df.isna().sum()}")

        # Volatility
        for window in [5, 10, 20]:
            df[f'Volatility_{window}'] = df['Return'].rolling(window).std() * np.sqrt(252)
        print("\n=== After Volatility ===")
        print(f"NaN Counts:\n{df.isna().sum()}")

        # On-Balance Volume (OBV)
        df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).cumsum()
        print("\n=== After OBV ===")
        print(f"NaN Counts:\n{df.isna().sum()}")

        # VWAP
        tp = (df['High'] + df['Low'] + df['Close']) / 3
        df['VWAP'] = (tp * df['Volume']).cumsum() / df['Volume'].cumsum()
        print("\n=== After VWAP ===")
        print(f"NaN Counts:\n{df.isna().sum()}")

        # Lagged Features
        for lag in [1, 2, 3, 5, 7, 10]:
            df[f'Return_Lag_{lag}'] = df['Return'].shift(lag)
            df[f'Volume_Lag_{lag}'] = df['Volume'].shift(lag)
        print("\n=== After Lagged Features ===")
        print(f"NaN Counts:\n{df.isna().sum()}")

        # Rolling Returns
        for window in [5, 10, 20]:
            df[f'Rolling_Return_{window}'] = df['Return'].rolling(window).mean()
        print("\n=== After Rolling Returns ===")
        print(f"NaN Counts:\n{df.isna().sum()}")

        # Price Momentum
        df['Momentum_5'] = df['Close'].pct_change(5)
        df['Momentum_10'] = df['Close'].pct_change(10)
        print("\n=== After Momentum ===")
        print(f"NaN Counts:\n{df.isna().sum()}")

        # Target Variable (Next period's return)
        df['Target'] = df['Close'].shift(-1).pct_change(fill_method=None).shift(1)  # Next day's return
        print("\n=== After Target ===")
        print(f"NaN Counts:\n{df.isna().sum()}")

        # Cleanup
        df = df.dropna(subset=['Target'])  # Only drop rows where 'Target' is NaN
        print("\n=== After dropna() ===")
        print(f"Shape: {df.shape}")
        print(f"NaN Counts:\n{df.isna().sum()}")

        df = df.loc[:, ~df.columns.duplicated()]
        print("\n=== After Removing Duplicate Columns ===")
        print(f"Shape: {df.shape}")
        print(f"NaN Counts:\n{df.isna().sum()}")

        print("\n=== After Feature Engineering ===")
        print(f"Shape: {df.shape}")
        print(f"NaN Counts:\n{df.isna().sum()}")
        print("Created features:", list(df.columns))
        
        return df

    except Exception as e:
        print(f"Feature engineering failed: {str(e)}")
        if 'df' in locals():
            print(f"NaN counts:\n{df.isna().sum()}")
            print(f"Columns: {df.columns.tolist()}")
        return None

# Data Preprocessing

In [33]:
def preprocess_data(df, test_size=0.2):
    """Splits and scales data with time-series awareness"""
    try:
        print("\nPreprocessing data...")
        
        # Features and target
        features = [col for col in df.columns if col not in ['Target', 'Close', 'Return']]
        X = df[features]
        y = df['Target']
        
        # Time-based split
        split_idx = int(len(X) * (1 - test_size))
        X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
        y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]
        
        # Scaling
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        return X_train_scaled, X_test_scaled, y_train, y_test, scaler
        
    except Exception as e:
        print(f"Preprocessing failed: {str(e)}")
        return None, None, None, None, None

# Model Training & Evaluation

In [34]:
def train_and_compare_models(X_train, X_test, y_train, y_test):
    """Trains and compares Linear Regression and Random Forest models"""
    try:
        print("\nTraining and comparing models...")
        results = {}
        
        # Linear Regression
        print("\n=== Linear Regression ===")
        lr = LinearRegression()
        lr.fit(X_train, y_train)
        lr_pred = lr.predict(X_test)
        
        results['Linear Regression'] = {
            'MAE': mean_absolute_error(y_test, lr_pred),
            'R²': r2_score(y_test, lr_pred),
            'RMSE': np.sqrt(mean_squared_error(y_test, lr_pred)),
            'model': lr
        }
        
        # Random Forest with Grid Search
        print("\n=== Random Forest ===")
        param_grid = {
            'n_estimators': [100, 200],
            'max_depth': [5, 10],
            'min_samples_split': [2, 4]
        }
        
        rf = RandomForestRegressor(random_state=42)
        grid_search = GridSearchCV(rf, param_grid, cv=TimeSeriesSplit(n_splits=3),
                                  scoring='neg_mean_absolute_error', verbose=0)
        grid_search.fit(X_train, y_train)
        best_rf = grid_search.best_estimator_
        rf_pred = best_rf.predict(X_test)
        
        results['Random Forest'] = {
            'MAE': mean_absolute_error(y_test, rf_pred),
            'R²': r2_score(y_test, rf_pred),
            'RMSE': np.sqrt(mean_squared_error(y_test, rf_pred)),
            'best_params': grid_search.best_params_,
            'model': best_rf
        }
        
        # Print comparison
        print("\nModel Comparison:")
        for model_name, metrics in results.items():
            print(f"\n{model_name}:")
            for k, v in metrics.items():
                if k != 'best_params' and k != 'model':
                    print(f"{k}: {v:.4f}")
            if 'best_params' in metrics:
                print(f"Best Params: {metrics['best_params']}")
        
        return results, lr_pred, rf_pred

    except Exception as e:
        print(f"Model training failed: {str(e)}")
        return None, None, None

# Backtest & Visualisations

In [35]:
def backtest_and_compare(data, X_test, lr_pred, rf_pred, y_test, risk_free_rate=0.0):
    """Performs detailed backtesting comparison"""
    try:
        print("\nStarting comparative backtest...")
        
        # Create base dataframe
        bt_df = data.loc[X_test.index, ['Close', 'Return']].copy()
        bt_df['Actual_Return'] = bt_df['Return']
        
        # Helper function to create strategy
        def create_strategy(pred_returns, name):
            df = bt_df.copy()
            df[f'Predicted_Return_{name}'] = pred_returns
            df[f'Signal_{name}'] = np.where(df[f'Predicted_Return_{name}'] > 0, 1, -1)
            df[f'Position_{name}'] = df[f'Signal_{name}'].diff().fillna(0).abs()
            
            # Transaction costs (0.1%)
            transaction_cost = 0.001
            df[f'Transaction_Cost_{name}'] = transaction_cost * df[f'Position_{name}'] * df['Close']
            
            # Strategy returns
            df[f'Strategy_Return_{name}'] = (df[f'Signal_{name}'] * df['Actual_Return']) - df[f'Transaction_Cost_{name}']
            df[f'Cumulative_{name}'] = (1 + df[f'Strategy_Return_{name}']).cumprod()
            
            return df

        # Create strategies
        lr_df = create_strategy(lr_pred, 'LR')
        rf_df = create_strategy(rf_pred, 'RF')
        
        # Combine results
        combined_df = pd.concat([lr_df, rf_df[['Cumulative_LR', 'Cumulative_RF']]], axis=1)
        combined_df['Cumulative_BuyHold'] = (1 + combined_df['Actual_Return']).cumprod()
        
        # Calculate metrics
        def calculate_metrics(returns, name):
            excess_returns = returns - risk_free_rate/252
            return {
                f'Total Return_{name}': returns.cumsum()[-1],
                f'Sharpe Ratio_{name}': excess_returns.mean() / excess_returns.std() * np.sqrt(252),
                f'Max Drawdown_{name}': (returns.cummax() - returns).max()
            }
        
        metrics = {}
        metrics.update(calculate_metrics(combined_df['Strategy_Return_LR'], 'LR'))
        metrics.update(calculate_metrics(combined_df['Strategy_Return_RF'], 'RF'))
        
        # Plot results
        plt.figure(figsize=(15, 10))
        plt.plot(combined_df['Cumulative_BuyHold'], label='Buy & Hold')
        plt.plot(combined_df['Cumulative_LR'], label='Linear Regression')
        plt.plot(combined_df['Cumulative_RF'], label='Random Forest')
        plt.title('Strategy Comparison: Cumulative Returns')
        plt.ylabel('Growth of $1')
        plt.legend()
        plt.grid(True)
        plt.show()
        
        return combined_df, metrics

    except Exception as e:
        print(f"Backtesting failed: {str(e)}")
        return None, None


# Main Script - move to main.py

In [36]:
if __name__ == "__main__":
    # Load and prepare data
    try:
        print("=== Data Loading ===")
        raw_data = load_data("data/KO_data.csv")
        
        if raw_data is not None:
            print("\n=== Feature Engineering ===")
            engineered_df = engineer_features(raw_data)
            
            if engineered_df is not None:
                # Stationarity check
                print("\n=== Stationarity Analysis ===")
                if not engineered_df.empty:
                    adfuller_result = adfuller(engineered_df['Close_diff'].dropna())
                    print(f"ADF Statistic: {adfuller_result[0]:.6f}")
                    print(f"p-value: {adfuller_result[1]:.6f}")
                    print("Critical Values:")
                    for key, value in adfuller_result[4].items():
                        print(f"\t{key}: {value:.3f}")
                else:
                    print("Error: Engineered DataFrame is empty. Cannot perform ADF test.")

                # Preprocess data
                print("\n=== Data Preprocessing ===")
                X_train, X_test, y_train, y_test, scaler = preprocess_data(engineered_df)
                
                if X_train is not None:
                    # Model training and comparison
                    print("\n=== Model Training ===")
                    model_results, lr_pred, rf_pred = train_and_compare_models(X_train, X_test, y_train, y_test)
                    
                    if model_results is not None:
                        # Backtesting and visualization
                        print("\n=== Backtesting ===")
                        backtest_df, metrics = backtest_and_compare(
                            engineered_df, 
                            X_test, 
                            lr_pred, 
                            rf_pred, 
                            y_test
                        )
                        
                        if backtest_df is not None:
                            # Display final results
                            print("\n=== Final Results ===")
                            print("\nModel Performance Summary:")
                            for model_name, metrics in model_results.items():
                                print(f"\n{model_name.upper()}:")
                                print(f"MAE: {metrics['MAE']:.4f}")
                                print(f"R²: {metrics['R²']:.4f}") 
                                print(f"RMSE: {metrics['RMSE']:.4f}")
                                if 'best_params' in metrics:
                                    print("Best Parameters:", metrics['best_params'])
                            
                            print("\nBacktest Metrics:")
                            print(pd.DataFrame.from_dict(metrics, orient='index').T.to_markdown())
                            
                            # Save results
                            print("\nSaving results...")
                            engineered_df.to_csv("processed_data.csv")
                            backtest_df.to_csv("backtest_results.csv")
                            print("Done!")
                            
    except Exception as e:
        print(f"Main execution failed: {str(e)}")
        print("Traceback:", traceback.format_exc())
    finally:
        print("\n=== Execution Complete ===")

=== Data Loading ===
Loading data from data/KO_data.csv...
Data loaded successfully
Data shape: (250, 6)
Data range: 2024-03-19 00:00:00 to 2025-03-18 00:00:00

=== Feature Engineering ===

=== Initial Data ===
Shape: (250, 6)
NaN Counts:
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

=== After Close_diff ===
NaN Counts:
Open          0
High          0
Low           0
Close         0
Adj Close     0
Volume        0
Close_diff    1
dtype: int64

=== After Return ===
NaN Counts:
Open          0
High          0
Low           0
Close         0
Adj Close     0
Volume        0
Close_diff    1
Return        0
dtype: int64

=== After Range ===
NaN Counts:
Open          0
High          0
Low           0
Close         0
Adj Close     0
Volume        0
Close_diff    1
Return        0
Range         0
dtype: int64

=== After Moving Averages ===
NaN Counts:
Open          0
High          0
Low           0
Close         0
Adj Close     0
Volume 

/Users/khalid/Desktop/Projects/Linear-Regression-Stock-Price-Movement-Prediction/venv/lib/python3.11/site-packages/sklearn/utils/extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Users/khalid/Desktop/Projects/Linear-Regression-Stock-Price-Movement-Prediction/venv/lib/python3.11/site-packages/sklearn/utils/extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/Users/khalid/Desktop/Projects/Linear-Regression-Stock-Price-Movement-Prediction/venv/lib/python3.11/site-packages/sklearn/utils/extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count
